In [ ]:
import pandas as pd
import os

In [ ]:
collection = []
for filename in os.listdir("."):
    if filename.endswith("_results.jsonl"):
        df = pd.read_json(filename, lines=True)
        df['model'] = filename.replace(".jsonl_results.jsonl", "").replace("samples_", "")
        collection.append(df)

df = pd.concat(collection)

In [ ]:
df

In [ ]:
df = df[df['model'] == 'gpt-4-1106-preview']

In [ ]:
# Group by task_id and count 'passed' status for each group
passed_counts = df.groupby('task_id')['passed'].sum()

# Display the result
passed_counts

In [ ]:
df1 = df[df['task_id'] == "./human-eval-bia/measure_0.ipynb"]
df2 = df1[df1['model'] == 'gpt-4-1106-preview']
for c, r in zip(df2['completion'], df2['result']):
    print('-----', r, ":")
    print(c)